In [1]:
# Basic Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For tesxt:
import re

# For times:
import time

# Set a random seed for imputation
#  Source:  https://numpy.org/doc/stable/reference/random/generated/numpy.random.seed.html
np.random.seed(42)

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer


# Read-in Data

In [2]:
# Import the Training Data
lstn = pd.read_csv('../data/listings_train.csv')

# Drop Un-needed Columns

In [3]:
lstn.drop(columns = [
    'listing_url', 'scrape_id', 'last_scraped', 'source',
    'picture_url', 'host_url', 'host_name', 'host_thumbnail_url', 'host_picture_url',
    'neighbourhood','neighbourhood_group_cleansed', 'minimum_minimum_nights',
    'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 
    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
    'calendar_last_scraped', 'bathrooms', 'first_review', 'last_review',
    'id', 'host_id',
], inplace = True)

# For now, these columsn will also be dropped unless time allows for them to be processed:
lstn.drop(columns = [
    'host_location', 'host_neighbourhood', 'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
    'review_scores_location', 'review_scores_value', 'license'
], inplace = True)

In [4]:
lstn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3477 entries, 0 to 3476
Data columns (total 41 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   name                                          3477 non-null   object 
 1   description                                   3464 non-null   object 
 2   neighborhood_overview                         2245 non-null   object 
 3   host_since                                    3477 non-null   object 
 4   host_about                                    2463 non-null   object 
 5   host_response_time                            3011 non-null   object 
 6   host_response_rate                            3011 non-null   object 
 7   host_acceptance_rate                          3067 non-null   object 
 8   host_is_superhost                             3476 non-null   object 
 9   host_listings_count                           3477 non-null   i

In [5]:
lstn.columns

Index(['name', 'description', 'neighborhood_overview', 'host_since',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object')

# Fix Datatypes

In [6]:
# FIX PRICE:  The dolar signs must be removed from the prices and numbers converted to float values
lstn.price = lstn.price.apply(lambda x: float(x.replace('$','').replace(',','').strip()))

In [7]:
# FIX HOST SINCE:  Convert to datetime then to epoch time in days

'''
The method used below to convert to epoch time was discovered with the help of ChatGPT.
Per the lead instructor, it is ok to use ChatGPT is a search tool provided that we provide the
question that was asked:

Question:  'in python, I want to convert a pandas datetime object to epoch time'

Additional help from:  https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
'''

# The strings are converted to date time, then to epoch time with '.timestamp()'
# The epoch time is then divided by the product the number of hours and seconds per day
#   to get the number of days since the epoch time origin 
lstn['host_since'] = pd.to_datetime(lstn['host_since']).apply(lambda x: x.timestamp()/(3600*24))

In [8]:
# FIX RESPONSE & ACCEPTANCE RATES:  Remove percentages

'''
Since the null values are not a very big percentage of the total data (though not a small percentage either),
the data will be imputed with the median value
'''

'''
The pcnt_floater functionwill be copied over here.  This is necessary as there are null values in these columns
which cannot be simply converted within the lambda function because there is no percentage sign.
'''

# This function will attempt to convert a string percentage value into a float
#  Source for help:  https://www.w3schools.com/python/python_try_except.asp
def pcnt_floater(x):
    try:
        return float(x.replace('%', '').strip())
    except:
        return x

# Convert percentages where they can be converted
lstn.host_acceptance_rate = lstn.host_acceptance_rate.apply(lambda x: pcnt_floater(x))
lstn.host_response_rate = lstn.host_response_rate.apply(lambda x: pcnt_floater(x))

# Impute Missing Data

In [9]:
# Impute missing text information with 'no_text_entered' into the following columns
nte_cols = ['description', 'neighborhood_overview','host_about', 'host_response_time']

for col in nte_cols:
    lstn[col].fillna('no_text_entered', inplace = True)

In [10]:
# Impute missing data with the median in the following columns
median_cols = ['host_response_rate', 'host_acceptance_rate', 'bedrooms', 'beds']

for col in median_cols:
    lstn[col].fillna(lstn[col].median(), inplace = True)

In [11]:
# Impute missing values with the mode in the following columns
lstn.host_is_superhost.fillna(lstn.host_is_superhost.mode()[0], inplace = True)
lstn.bathrooms_text.fillna(lstn.bathrooms_text.mode()[0], inplace = True)

In [12]:
# Impute missing data with 0 in reviews per month
lstn.reviews_per_month.fillna(0, inplace = True)

In [13]:
# Verify imputation
sum(lstn.isnull().sum() != 0)

0

# Create Simple Numerical Features

In [14]:
# Create percentage columns for the calculated listings by listing type
lstn['pcnt_ent_homes'] = round(lstn['calculated_host_listings_count_entire_homes'] / lstn['calculated_host_listings_count'], 3)
lstn['pcnt_private'] = round(lstn['calculated_host_listings_count_private_rooms'] / lstn['calculated_host_listings_count'], 3)
lstn['pcnt_shared'] = round(lstn['calculated_host_listings_count_shared_rooms'] / lstn['calculated_host_listings_count'], 3)

## Add T-Stop Distnace Data

In [15]:
# Note, this file has been added to the gitignore file and is NOT located in the repository
geodata = pd.read_csv('../data/lat_lng_data.csv')

latitudes = list(geodata.lat)
longitudes = list(geodata.long)

In [16]:
# This funciton was written around the following source:
# https://towardsdatascience.com/create-new-column-based-on-other-columns-pandas-5586d87de73d

def min_dist(fn_lat, fn_lng, lat_data, lng_data):
    
    # Set a minimum distnace well beyond anything that would be derived
    min_dist = 90
    
    # Write a loop to find the minimum (euclidean) distance to every T-stop
    for n in range(len(latitudes)):
        dist = ((fn_lat - lat_data[n])**2 + (fn_lng - lng_data[n])**2)**0.5
        
        # Store this distance if smaller than min distance
        if dist < min_dist:
            min_dist = dist
    
    return min_dist

In [17]:
# Crate a new column with the minimum distance to any T-stop
#  The following source was used to help write this code (note axis = 1 is KEY!):
    # https://towardsdatascience.com/create-new-column-based-on-other-columns-pandas-5586d87de73d
lstn['min_distance'] = lstn.apply(lambda x: min_dist(x.latitude, x.longitude, latitudes, longitudes), axis = 1)

# Create Log Features

In [18]:
'''
The col_logger function will need to be brough in from the other notebooks

This has been modified to include a 0 imputation value n such that transforamtion
occurs on log(n) and not log(0) which is undefined.
'''

def col_logger(data_column, zero_imp = 1):
    # Since log(0) is undefined, 0's must be treated as log(1)
    return data_column.apply(lambda x: np.log(zero_imp) if x==0 else np.log(x))

In [19]:
lstn._get_numeric_data().describe().T

,count,mean,std,min,25%,50%,75%,max
host_since,3477.0,17138.849871,1079.936868,14216.000000,16352.000000,17151.000000,18090.000000,19422.000000
host_response_rate,3477.0,97.487489,9.035327,0.000000,100.000000,100.000000,100.000000,100.000000
host_acceptance_rate,3477.0,88.469083,20.483848,0.000000,92.000000,97.000000,100.000000,100.000000
host_listings_count,3477.0,478.265746,1349.140443,1.000000,2.000000,12.000000,84.000000,4807.000000
host_total_listings_count,3477.0,636.246189,1517.960227,1.000000,3.000000,17.000000,127.000000,5358.000000
latitude,3477.0,42.337493,0.027111,42.235300,42.321870,42.344720,42.354310,42.392280
longitude,3477.0,-71.082429,0.033393,-71.173486,-71.100490,-71.073160,-71.060810,-70.996000
accommodates,3477.0,3.181478,2.206180,1.000000,2.000000,2.000000,4.000000,16.000000
bedrooms,3477.0,1.659477,1.155218,1.000000,1.000000,1.000000,2.000000,13.000000
beds,3477.0,1.777107,1.427401,1.000000,1.000000,1.000000,2.000000,22.000000


In [20]:
# Create a list of numerical columns
num_cols = list(lstn._get_numeric_data().columns)

# Remove latitiude and longitude data as they were used previously to create distances
num_cols.remove('latitude')
num_cols.remove('longitude')

In [21]:
for col in num_cols:
    
    # Find columsn with values between 0 and 1
    if len(lstn[col][(lstn[col] < 1) & (lstn[col] > 0)]) > 0:
        
        # Determine the minimum value in that column, if it's 0, base the minimum
        #  value off of the second smallest value in the column
        
        if min(lstn[col]) < 0:
            print('CANT LOGARITHM A NEGATIVE NUMBER')
            break
        
        elif min(lstn[col]) == 0:
            # second smallest value
            min_col_val = lstn[col].sort_values().unique()[1]
        
        else:
            min_col_val = min(lstn[col][(lstn[col] < 1) & (lstn[col] > 0)])
        
        # Calculate a zero imputation value for use in the col_logger function
        #  Take the natural log of the minimum value and round down
        '''
        This last step ensures that any zero values will be less than any positive
        values after a log transformation.
        '''
        z_imp = np.exp(np.floor(np.log(min_col_val)))
        
        # Transform the column:
        lstn[f'log_{col}'] = col_logger(lstn[col], z_imp)
    
    else:
        # Otherwise, simply use defulat zero_imputation value of 1
        lstn[f'log_{col}'] = col_logger(lstn[col])

In [26]:
# Check for null values to verify proper feature creation
sum(lstn.isnull().sum())

0

# One Hot Encoding

In [33]:
# Find the remaining categorical columns:
cat_cols = list(set(lstn.columns) - set(lstn._get_numeric_data().columns))
cat_cols

['neighbourhood_cleansed',
 'host_is_superhost',
 'has_availability',
 'room_type',
 'property_type',
 'amenities',
 'host_verifications',
 'host_response_time',
 'host_about',
 'name',
 'bathrooms_text',
 'host_has_profile_pic',
 'host_identity_verified',
 'instant_bookable',
 'neighborhood_overview',
 'description']

In [34]:
# Some of these columns must be removed to be handled separately in advanced processing
remove_cols = ['amenities', 'host_about', 'name', 'neighborhood_overview', 'description']

for col in remove_cols:
    cat_cols.remove(col)
    
cat_cols

['neighbourhood_cleansed',
 'host_is_superhost',
 'has_availability',
 'room_type',
 'property_type',
 'host_verifications',
 'host_response_time',
 'bathrooms_text',
 'host_has_profile_pic',
 'host_identity_verified',
 'instant_bookable']

In [43]:
# Merge the lstn dataframe with the one hot encoded values
lstn = pd.get_dummies(lstn, columns = cat_cols, drop_first=True)

In [44]:
lstn.shape

(3477, 160)

# Advanced Processing

## Amenities

In [49]:
# Use a regular expression to extract the amenities which are between quotes.
#  Code adapted from this source: https://stackoverflow.com/questions/1454913/regular-expression-to-find-a-string-included-between-two-characters-while-exclud
# Also helpful:  https://regex101.com/
regex_string = '(?<=")[^"]+(?="[,\]])'

amn_lst = []

for string_lists in lstn.amenities:
    a_list = re.findall(regex_string, string_lists)
    for amenity in a_list:
        amn_lst.append(amenity)
        


In [64]:
# Create a pandas series of all amenities and their number of occurences
amn_counts = pd.Series(amn_lst).value_counts(ascending=False)

# Filter the datafarme to use only words that appear in 99% of posts
#  THIS IS REQUIRED GIVEN THAT min_df IS IGNORED BY COUNT VECTORISZER WITH CUSTOM DICTIONARIES
#  Create a vocab variable by using the index attribute to get the list of amenities
amn_vocab = amn_counts[amn_counts >= 35].index

In [66]:
# Use countevectorizer to one hot encode all the amenities
#  Use the vocab to get only the amenities encoded
#  NOTE:  Set the 'token_pattern' to the regex string so it finds the exact same tokens as were found previously
cvec = CountVectorizer(lowercase=False,
                       vocabulary=amn_vocab,
                       ngram_range=(1, 1),
                       token_pattern=regex_string,
                      )

# Create a new dataframe with the count vectorized data from the amenities column
amen_df = pd.DataFrame(cvec.fit_transform(lstn.amenities).todense(), 
             columns = cvec.get_feature_names_out())

amen_df

,Wifi,Smoke alarm,Carbon monoxide alarm,Kitchen,Essentials,Hangers,Iron,Hair dryer,Heating,Microwave,...,Paid parking lot off premises,Park view,Mosquito net,Pool table,Heating - split type ductless system,Smoking allowed,Free driveway parking on premises \u2013 1 space,Stainless steel electric stove,HDTV,Baby safety gates
0,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3472,1,1,1,1,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3473,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3474,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3475,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Create the amenities count columns and log of that column

In [74]:
# Using regex as before, find the number of amenities for every listing and store to a new column
lstn['amen_cnt'] = lstn.amenities.apply(lambda x: len(re.findall(regex_string, x)))

# Create a log transformed column, setting the zero imputation value to e^-1
lstn['log_amen_cnt'] = col_logger(lstn['amen_cnt'], np.exp(-1))

## Then merge the amenity dataframe into the lstn data frame
### YOu'll first want to rename the columns by manually adding a prefix

In [41]:
# SVAE IT FOR LATER!!!!


pd.merge(left = lstn, 
         right = pd.get_dummies(lstn[cat_cols], drop_first=True), 
         left_index=True, right_index=True, 
         how = 'outer')

,name,description,neighborhood_overview,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,...,bathrooms_text_4 baths,bathrooms_text_4 shared baths,bathrooms_text_4.5 baths,bathrooms_text_5 baths,bathrooms_text_6 baths,bathrooms_text_Half-bath,bathrooms_text_Private half-bath,host_has_profile_pic_t,host_identity_verified_t,instant_bookable_t
0,"Private, Quiet, Comfy, Clean Apartment in Boston","This is the 2 bedroom. It’s one bathroom, gian...",no_text_entered,16362.0,no_text_entered,no_text_entered,100.0,97.0,f,1,...,0,0,0,0,0,0,0,1,1,1
1,"Renovated, Clean, Quiet & Safe Near Orange Line",Solid and fast wifi. Close to Stony Brook stat...,Peaceful and orderly neighborhood. Near the ho...,16272.0,no_text_entered,within an hour,100.0,0.0,f,31,...,0,0,0,0,0,0,0,1,1,0
2,Separate entrance in-law suite in heart of JP,"Pet friendly, sunny, separate entrance, in law...",Jamaica Plain has copious green space (Arnold ...,17390.0,I am an artist and writer. I teach visual art ...,no_text_entered,100.0,100.0,f,2,...,0,0,0,0,0,0,0,1,1,1
3,Gorgeous Studio with Bay Windows!,Stay in the heart of Boston’s historic and pop...,South End is a modern and trendy hotspot of Bo...,18806.0,Hey we're Justin and Mandy! We love hockey and...,within an hour,100.0,99.0,f,38,...,0,0,0,0,0,0,0,1,0,1
4,Modern 1BR | Kitchen + W/D | Free WiFi + Desk,What’s more exciting than the latest sell-out ...,"Centrally located to shopping, dining, and wor...",16524.0,no_text_entered,within an hour,99.0,95.0,f,540,...,0,0,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3472,"Sunny, upper floor facing the State House!",Live beside the State House and the Boston Com...,Beacon Hill is a picturesque neighborhood fill...,14447.0,I manage a number of furnished units around Bo...,within a day,82.0,55.0,f,12,...,0,0,0,0,0,0,0,1,1,0
3473,"Furnished , Newly Renovated, Spacious Studio","Beautiful, modern, renovated studio found on t...","Quiet tree-lined street, but steps to restaura...",18051.0,Welcome to STARS of Boston (Short Term Apartme...,within an hour,100.0,96.0,f,60,...,0,0,0,0,0,0,0,1,1,0
3474,"Room near MBTA transit, Keyless self check-in","Comfortable, spacious private room in the quie...",no_text_entered,17724.0,no_text_entered,within a few hours,100.0,100.0,t,7,...,0,0,0,0,0,0,0,1,0,0
3475,Furnished Queen Bedroom G in #1321: Allston,"This comfy, cute and charming room is availabl...",no_text_entered,18164.0,"Ready, set, rent! Discover a rental experience...",within a few hours,95.0,51.0,f,320,...,0,0,0,0,0,0,0,1,1,1


In [23]:
for col in lstn._get_numeric_data().columns:
    if len(lstn[col][(lstn[col] < 1) & (lstn[col] > 0)]) > 0:
        print(col)
        
        min_vally = min(lstn[col][(lstn[col] < 1) & (lstn[col] > 0)])
        print(min_vally)
        print(np.exp(np.floor(np.log(min_vally))))

reviews_per_month
0.01
0.006737946999085467
pcnt_ent_homes
0.018
0.006737946999085467
pcnt_private
0.04
0.01831563888873418
pcnt_shared
0.032
0.01831563888873418
min_distance
8.421787219885861e-05
4.5399929762484854e-05
log_host_listings_count
0.6931471805599453
0.36787944117144233
log_host_total_listings_count
0.6931471805599453
0.36787944117144233
log_accommodates
0.6931471805599453
0.36787944117144233
log_bedrooms
0.6931471805599453
0.36787944117144233
log_beds
0.6931471805599453
0.36787944117144233
log_minimum_nights
0.6931471805599453
0.36787944117144233
log_maximum_nights
0.6931471805599453
0.36787944117144233
log_availability_30
0.6931471805599453
0.36787944117144233
log_availability_60
0.6931471805599453
0.36787944117144233
log_availability_90
0.6931471805599453
0.36787944117144233
log_availability_365
0.6931471805599453
0.36787944117144233
log_number_of_reviews
0.6931471805599453
0.36787944117144233
log_number_of_reviews_ltm
0.6931471805599453
0.36787944117144233
log_number_of